# NSWEduChat Translations & Post-Edited Data Processing

This notebook performs the following steps:
1. **Importing Libraries & Reading Data:**  
   It imports necessary libraries and reads two Excel files containing translation data and post-edited texts.
   
2. **Extracting & Saving Translations:**  
   It extracts translation columns from the Excel sheets, creates a combined DataFrame, and writes individual language translations as text files. In addition, a complete CSV file of all translations is created.
   
3. **Processing Post-Edited Data:**  
   The notebook reads a second Excel file (with sentence-level and paragraph-level texts) and separates sheets based on whether they contain “sentence” or “paragraph” data.  
   For each sheet, it:
   - Flags rows that contain the word "Translation" (or empty strings) in the `Original` column.
   - Selects only those rows that are not flagged.
   - Extracts the `Generated` (original) and `Corrected` (post-edited) texts.
   - Drops any missing values and resets the index.
   - Saves the processed data to CSV files in separate directories for paragraphs and sentences.
   - Finally, a JSON file is written listing all processed language names for each category.

In [1]:
import pandas as pd
import json

In [2]:
sheets_dict = pd.read_excel(
    "../data/NSWEduChat_all languages_60 translations[100].xlsx",
    engine="openpyxl",
    sheet_name=None,
)

In [3]:
data = pd.DataFrame()

In [4]:
data["english"] = sheets_dict["Arabic"]["English paragraph"]

In [5]:
for lang in sheets_dict.keys():
    data[lang.lower()] = sheets_dict[lang]["NSWEduChat translation"]

In [6]:
for lang in data.columns:
    data[lang].to_csv(
        f"../data/educhat-translation/{lang.lower()}.txt", index=False, header=False
    )

In [7]:
data.to_csv("../data/educhat-translation/all_translations.csv", index=False)

## Processing Post-Edited Data

In the next section, we process a second Excel file that contains both sentence-level and paragraph-level data along with their post-edited versions.

- **Sheet Separation:**  
  We read the Excel file and then separate the sheet names based on whether they end with "sentence" or "paragraph".

- **Post-Editing Processing:**  
  For each sheet, we:
  - Create a flag to mark rows where the `Original` column either contains the word "Translation" or is empty.
  - Filter out these flagged rows.
  - Extract the `Generated` (from the `Original` column) and `Corrected` (from the `MNSW_post-edited` column) texts.
  - Drop missing values and reset the index.
  - Write the cleaned data to CSV files in appropriate directories.
  - Collect the language names processed and save them to JSON files.

In [8]:
sheets_dict = pd.read_excel(
    "../data/Languages_by_sentence and paragraph.xlsx",
    engine="openpyxl",
    sheet_name=None,
)

sheets_dict["Arabic_full paragraph"]

,Original,MNSW_post-edited
0,Translation 1,Translation 1
1,\nعزيزي أولياء الأمور/الوصيين، أتمنى أن تكونوا...,\nأعزائي الأهل/أولياء الأمور،\nأتمنى أن تكونوا...
2,Translation 2,Translation 2
3,\n نحن متحمسون للإعلان أن فرقة الرقص الموهوبة ...,\n يسرنا الإعلان عن أن فرقة الرقص الموهوبة لد...
4,Translation 3,Translation 3
5,\n العنصر 25 أولياء الأمور والأوصياء الأعزاء، ...,\nأعزائي الأهل وأولياء الأمور، \nيسعدنا جداً أ...
6,Translation 4,Translation 4
7,\n البند 33 التحيات: مرحبا، أهلاً، هيا، صباح ا...,\nالتحيات: مرحباً، مرحباً، مرحباً، صباح الخير،...
8,Translation 5,Translation 5
9,\n الطفل 1: مرحبًا، أنا ليلي. ما اسمك؟ الطفل 2...,\n الطفلة الأول: مرحبًا، أنا ليلي. ما اسمك؟ ال...


In [9]:
sentences = [key for key in sheets_dict.keys() if key.split()[-1] == "sentence"]
paragraphs = [key for key in sheets_dict.keys() if key.split()[-1] == "paragraph"]
paragraphs

['Arabic_full paragraph',
 'Bengali_full paragraph',
 'Burmese_full paragraph',
 'Cantonese_full paragraph',
 'Hindi_full paragraph',
 'Indonesian_full paragraph',
 'Japanese_full paragraph',
 'Khmer_full paragraph',
 'Mandarin_full paragraph',
 'Mongolian_full paragraph',
 'Nepali_full paragraph',
 'Persian_full paragraph',
 'Portuguese_full paragraph',
 'Punjabi_full paragraph',
 'Spanish_full paragraph',
 'Swahili_full paragraph',
 'Tagalog_full paragraph',
 'Tamil_full paragraph',
 'Thai_full paragraph',
 'Turkish_full paragraph',
 'Urdu_full paragraph',
 'Vietnamese_full paragraph']

In [10]:
par_langs = []

for lang in paragraphs:
    entry = dict()
    data = pd.DataFrame()
    sheets_dict[lang]["flag"] = sheets_dict[lang]["Original"].apply(
        lambda x: (
            "Translation" in x.split() or len(x.strip().split()) == 0
            if isinstance(x, str)
            else False
        )
    )
    data["Generated"] = sheets_dict[lang]["Original"].loc[~sheets_dict[lang]["flag"]]
    data["Corrected"] = sheets_dict[lang]["MNSW_post-edited"].loc[
        ~sheets_dict[lang]["flag"]
    ]
    data = data.dropna()
    data = data.reset_index(drop=True)
    data.to_csv(
        f"../data/post-edited/paragraph/{lang.replace('_', ' ').split()[0].lower()}.csv",
        index=False,
    )
    par_langs.append(lang.replace("_", " ").split()[0].lower())


with open("../data/post-edited/paragraph.json", "w") as json_file:
    json.dump(par_langs, json_file, indent=4)

In [11]:
sent_langs = []

for lang in sentences:
    data = pd.DataFrame()
    sheets_dict[lang]["flag"] = sheets_dict[lang]["Original"].apply(
        lambda x: (
            "Translation" in x.split() or len(x.strip().split()) == 0
            if isinstance(x, str)
            else False
        )
    )
    data["Generated"] = sheets_dict[lang]["Original"].loc[~sheets_dict[lang]["flag"]]
    data["Corrected"] = sheets_dict[lang]["MNSW_post-edited"].loc[
        ~sheets_dict[lang]["flag"]
    ]
    data = data.dropna()
    data = data.reset_index(drop=True)
    data.to_csv(
        f"../data/post-edited/sentence/{lang.replace('_', ' ').split()[0].lower()}.csv",
        index=False,
    )
    sent_langs.append(lang.replace("_", " ").split()[0].lower())

with open("../data/post-edited/sentence.json", "w") as json_file:
    json.dump(sent_langs, json_file, indent=4)